In [1]:
import matplotlib
import sys
import os
from statsmodels.tsa.ar_model import AutoReg, ar_select_order, AutoRegResults
from scipy.stats import norm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

matplotlib.rcParams.update({'font.size': 25})


def ramp(x):

    if x > 0:

        return x

    else:

        return 0.0

if __name__=='__main__':


            # Parameters
            sampling_rate = 100  # samples per second
            total_time = 3  # seconds
            num_samples = total_time * sampling_rate
            time = np.arange(num_samples) / sampling_rate

            # Generate sine waves
            sine_wave_1 = np.sin(2 * np.pi * 10 * time[:sampling_rate])
            sine_wave_2 = np.sin(2 * np.pi * 20 * time[sampling_rate:2 * sampling_rate])
            sine_wave_3 = np.sin(2 * np.pi * 30 * time[2 * sampling_rate:])

            # Combine the sine waves
            signal = np.concatenate([sine_wave_1, sine_wave_2, sine_wave_3])

            # Add Gaussian noise
            noise = 0.1 * np.random.randn(num_samples)
            noisy_signal = signal + noise

            # Some initial parameters
            for order in [8]: # n of orders in AR model

                # Some model parameters...
                bn = 4*order # burn in data
                e_t = 0.0 # initialising the prediction error
                pred_y_t = 0.0 # initialising the predicted state
                sigma2_ypred = 0.0
                smooth_param = 0.1
                state_noise_var = 0.0

                # Split data
                # train = df_polars_raw_pd_price_last['ret'][:bn].values
                # test = df_polars_raw_pd_price_last['ret'][bn:].values
                train = noisy_signal[:bn]
                test = noisy_signal[bn:]

                # Parameters for state noise
                # state_noise_t = 0.0 # initialising the state noise (SHOULD BE A MATRIX)
                W_t = np.zeros((order, order)) #np.cov(state_noise_t) # initialising the state noise covariance matrix (SHOULD BE A MATRIX)
                state_noise_coeff_matrix_tminus1 = np.eye(order) # np.linalg.cholesky(W_t).T # initialising the state noise coefficient matrix (SHOULD BE A MATRIX)
                R_tminus1 = np.eye(order)

                ## Start the iteration
                state_noise_arr = [] # collecting the state noise variance
                w_arr = [] # collecting the state noise covariance
                e_arr = [] # collecting the prediction error
                k_arr = [] # collecting the kalman gain
                sigma2_ypred_arr = [] # collecting the estimated prediction variance
                q_cov_tminus1_arr = [] # collecting the cov matrix of the predicted state
                pred_y_arr = []
                pred_q_arr = []
                post_arr = []
                rho_arr = []
                rho_arr_complete = []
                sigma2_q0_arr = []

                for n, i in enumerate(range(order+1, len(test) + 1)):

                    # Compute the vector of observations
                    smallerDF_ret = test[i - (order+1):(i)]
                    smallerDF_arr = smallerDF_ret[:-1]
                    next_data_pt = smallerDF_ret[-1] # t
                    F_t = -1 * smallerDF_arr[::-1] # collecting observations from t-1 to t-p

                    # print(F_t)

                    if n == 0:

                        # Obtain the initial estimate for predicted state, their covariance and the observation noise variance
                        model_init = AutoReg(train[::-1], trend='n', lags=order).fit()
                        pred_q = model_init.params  # initial estimates of state vector
                        # sigma2_0 = abs(np.var(F_t))
                        #sigma2_0 = np.average([abs(np.var(test[0:10])), abs(np.var(test[int(len(test)/2):int(len(test)/2+10)])), abs(np.var(test[int(len(test)-10):]))])# take in several overlapping blocks of data and compute the variance of the data
                        sigma2_0 = 0.2#0.1**2.
                        # sigma2_0 = abs(np.var(model_init.resid))
                        # sigma2_0 = abs(np.var(train))
                        # inv_train = -1 * train[::-1]
                        # q_cov_tminus1 = sigma2_0 * np.dot(inv_train, inv_train.transpose())
                        ## q_cov_tminus1 = sigma2_0 * np.dot(F_t, F_t.transpose()) # initial state covariance matrix set to the linear regression covariance matrix (p9 penny)
                        q_cov_tminus1 = sigma2_0 * np.einsum("i,j->ij", F_t, F_t)
                        # q_cov_tminus1 = sigma2_0 * np.einsum("i,i", F_t, F_t) * np.eye(order)
                        #print(q_cov_tminus1.shape)

                        # Estimate the state noise on-line using Jazwinski algorithm (adaptive kalman filtering); using N (lag/burnin) = 1
                        # C = np.einsum("i,ij->j", F_t, q_cov_tminus1)
                        # D = np.einsum("i,i", C, F_t)
                        # sigma2_q0 = sigma2_0 + D # Compute the estimated predicted variance assuming q=0
                        # state_noise_var = ((e_t ** 2) - sigma2_q0) / np.einsum("i,i", F_t, F_t)

                    if n > 0:


                        # # Estimate the k+1 state noise from k+l where l = 1 on-line using Jazwinski algorithm (adaptive kalman filtering)
                        # sigma2_q0 = sigma2_0 + D # Compute the estimated predicted variance assuming q=0
                        # new_estimate = ((e_t ** 2) - sigma2_q0) / np.einsum("i,i", F_t, F_t)
                        # # state_noise_var = new_estimate
                        # state_noise_var = new_estimate #smooth_param * state_noise_var + (1-smooth_param) * new_estimate # updating the state noise variance for next step
                        # # print(state_noise_var)
                        # state_noise_arr.append(state_noise_var)

                        # Compute the predicted observation (mean of the likelihood distribution)
                        # we need to / can do this first as we use pred_q from tminus1 to compute pred_y_t
                        pred_y_t = np.dot(F_t, pred_q)
                        pred_y_arr.append(pred_y_t)

                        # Compute the prediction error (e_t)
                        e_t = next_data_pt - pred_y_t
                        e_arr.append(e_t**2)

                        # Updating the observation noise
                        # E = np.einsum("i,ij->j", F_t, R_tminus1)
                        # F = np.einsum("i,i", E, F_t)
                        # sigma2_0 = smooth_param* sigma2_0 + (1-smooth_param) * (e_t**2 - F)

                        # Estimate the k+1 state noise from k+l where l = 1 on-line using Jazwinski algorithm (adaptive kalman filtering)
                        C = np.einsum("i,ij->j", F_t, q_cov_tminus1)
                        D = np.einsum("i,i", C, F_t)
                        sigma2_q0 = sigma2_0 + D # Compute the estimated predicted variance assuming q=0
                        sigma2_q0_arr.append(sigma2_q0)
                        new_estimate = ((e_t ** 2) - sigma2_q0) / np.einsum("i,i", F_t, F_t)
                        new_estimate = ramp(new_estimate)
                        state_noise_var = smooth_param * state_noise_var + (
                                    1 - smooth_param) * new_estimate  # when residuals become large relative to their predicted 1sigma limit, filter is diverging

                         # updating the state noise variance for next step
                        state_noise_arr.append(state_noise_var)

                        # Compute the state noise covariance matrix using state noise variance
                        W_t = state_noise_var * np.eye(order)
                        w_arr.append(np.linalg.norm(W_t, ord=2))

                        # Compute estimated prediction variance (E11)
                        R_t = q_cov_tminus1 + W_t
                        J = np.einsum("i,ij->j", F_t, R_t)
                        sigma_2phi = np.einsum("i,i", J, F_t)
                        sigma2_ypred = sigma2_0 + sigma_2phi
                        sigma2_ypred_arr.append(sigma2_ypred)

                        # Compute the Kalman gain (E10)
                        K_t = np.einsum("ij,j->i", R_t, F_t) / sigma2_ypred
                        #K_t = np.dot((q_cov_tminus1 + W_t), F_t.transpose()) / sigma2_ypred
                        k_arr.append(K_t[0])

                        # Compute the learning rate rho
                        rho = (1./order) * np.matrix.trace(R_t)
                        ## rho_arr.append(rho)
                        rho = np.matmul((1/order) * (R_t), F_t.transpose())
                        # rho2 = np.matmul(rho, F_t.transpose())
                        rho_arr.append(rho.sum())

                        rho_c = ((1. / order) * np.trace(R_t))/ sigma2_ypred
                        # rho_arr_complete.append(rho_c)
                        #rho_c = np.matmul((((1. / order) * (R_t))/ sigma2_ypred), F_t.transpose())
                        # rho_c2 = np.matmul(rho_c, F_t.transpose())
                        rho_arr_complete.append(rho_c)#.sum())

                        # Compute the posterior probability of P(y_t) [EQN 12]
                        posterior = norm.pdf(next_data_pt, loc=pred_y_t, scale=np.sqrt(sigma2_ypred)) # computing the posterior probability of seeing y_t
                        post_arr.append(posterior)
                        #raise SystemExit(0)

                        #################################################################################

                        ### GETTING READY FOR THE NEXT ITERATION ###
                        # Compute the covariance of the state estimate (E9)
                        A = np.einsum("i,j->ij", K_t, F_t)
                        B = np.einsum("ij,jk->ik", A, R_t)
                        q_cov_tminus1 = R_t - B # updating the state covariance matrix for next step
                        # print(q_cov_tminus1)
                        q_cov_tminus1_arr.append(q_cov_tminus1[0,0])

                        # if(state_noise_var==math.inf):
                        #     state_noise_var=0.0

                        # Update the predicted state (E8)
                        pred_q = pred_q + (K_t * e_t) # updating predicted state for the next iteration
                        pred_q_arr.append(pred_q[0])

                        # R_tminus1 = R_t

ModuleNotFoundError: No module named 'statsmodels'